<b>Agenda</b>
1) Using pipelines
<br>
2) Construct plots
<br>
3) Improve performance
<br>
4) Push code into Github
<br>
<br>
<b>Long-term goals</b>
1) Loading data in batches
<br>
2) Parallel processing (perhaps using Apache Spark)

In [1]:
import pandas as pd
import tensorflow as tf
import os
import numpy as np
import math
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
dataset_df = pd.read_csv('train.csv')

# 1) View summary of the data

In [3]:
print(f'Shape of the training data = ', dataset_df.shape)

Shape of the training data =  (5237980, 17)


In [4]:
print('First 5 rows: -')
dataset_df.head()

First 5 rows: -


,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


In [5]:
print('Description of the data: -')
dataset_df.describe()

Description of the data: -


,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id
count,5.237980e+06,5.237980e+06,5.237980e+06,5.237760e+06,5.237980e+06,5.237760e+06,5.237760e+06,2.343638e+06,2.380800e+06,5.237760e+06,5.237980e+06,5.237760e+06,5.237980e+06,5.237760e+06,5.237892e+06,5.237980e+06
mean,9.928856e+01,2.415100e+02,2.700000e+02,5.715293e+06,-1.189619e-02,9.999955e-01,4.510025e+07,1.001713e+00,9.996601e-01,9.997263e-01,5.181359e+04,1.000264e+00,5.357568e+04,9.999920e-01,-4.756125e-02,1.331005e+04
std,5.787176e+01,1.385319e+02,1.587451e+02,2.051591e+07,8.853374e-01,2.532497e-03,1.398413e+08,7.214705e-01,1.216920e-02,2.499345e-03,1.114214e+05,2.510042e-03,1.293554e+05,2.497509e-03,9.452860e+00,7.619271e+03
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,9.352850e-01,4.316610e+03,7.700000e-05,7.869880e-01,9.349150e-01,0.000000e+00,9.398270e-01,0.000000e+00,9.380080e-01,-3.852898e+02,0.000000e+00
25%,4.900000e+01,1.220000e+02,1.300000e+02,8.453415e+04,-1.000000e+00,9.987630e-01,5.279575e+06,9.963320e-01,9.971000e-01,9.985290e-01,7.374720e+03,9.990290e-01,7.823700e+03,9.987810e-01,-4.559755e+00,6.729000e+03
50%,9.900000e+01,2.420000e+02,2.700000e+02,1.113604e+06,0.000000e+00,9.999670e-01,1.288264e+07,9.998830e-01,9.998890e-01,9.997280e-01,2.196900e+04,1.000207e+00,2.301792e+04,9.999970e-01,-6.020069e-02,1.334500e+04
75%,1.490000e+02,3.610000e+02,4.100000e+02,4.190951e+06,1.000000e+00,1.001174e+00,3.270013e+07,1.003318e+00,1.002590e+00,1.000905e+00,5.583168e+04,1.001414e+00,5.787841e+04,1.001149e+00,4.409552e+00,1.990700e+04
max,1.990000e+02,4.800000e+02,5.400000e+02,2.982028e+09,1.000000e+00,1.077488e+00,7.713682e+09,4.379531e+02,1.309732e+00,1.077488e+00,3.028784e+07,1.077836e+00,5.440500e+07,1.077675e+00,4.460704e+02,2.645400e+04


# 2) Handling missing values

## 2.1) Determine missingness in the dataframe

In [6]:
print("List of missing values for various columns of the dataframe: -")
num_missing_vals_series = (dataset_df.isnull().sum(axis = 0)/dataset_df.shape[0]) * 100
num_missing_vals_series

List of missing values for various columns of the dataframe: -


stock_id                    0.000000
date_id                     0.000000
seconds_in_bucket           0.000000
imbalance_size              0.004200
imbalance_buy_sell_flag     0.000000
reference_price             0.004200
matched_size                0.004200
far_price                  55.256836
near_price                 54.547364
bid_price                   0.004200
bid_size                    0.000000
ask_price                   0.004200
ask_size                    0.000000
wap                         0.004200
target                      0.001680
time_id                     0.000000
row_id                      0.000000
dtype: float64

In [7]:
index = 0

for feature in num_missing_vals_series.index:
    
    if (num_missing_vals_series[feature] > 0) and (num_missing_vals_series[feature] < 1):
        print(f'Feature = {feature}')
        print(f'Percentage of missing values = {num_missing_vals_series[feature]}')
        print(f'Dropping missing values for {feature}')
        dataset_df.dropna(subset = feature, inplace = True)
        print('\n')
    
    index += 1

Feature = imbalance_size
Percentage of missing values = 0.004200092402032845
Dropping missing values for imbalance_size


Feature = reference_price
Percentage of missing values = 0.004200092402032845
Dropping missing values for reference_price


Feature = matched_size
Percentage of missing values = 0.004200092402032845
Dropping missing values for matched_size


Feature = bid_price
Percentage of missing values = 0.004200092402032845
Dropping missing values for bid_price


Feature = ask_price
Percentage of missing values = 0.004200092402032845
Dropping missing values for ask_price


Feature = wap
Percentage of missing values = 0.004200092402032845
Dropping missing values for wap


Feature = target
Percentage of missing values = 0.0016800369608131378
Dropping missing values for target




In [8]:
num_missing_vals_series = (dataset_df.isnull().sum(axis = 0)/dataset_df.shape[0]) * 100
num_missing_vals_series

stock_id                    0.000000
date_id                     0.000000
seconds_in_bucket           0.000000
imbalance_size              0.000000
imbalance_buy_sell_flag     0.000000
reference_price             0.000000
matched_size                0.000000
far_price                  55.254956
near_price                 54.545455
bid_price                   0.000000
bid_size                    0.000000
ask_price                   0.000000
ask_size                    0.000000
wap                         0.000000
target                      0.000000
time_id                     0.000000
row_id                      0.000000
dtype: float64

In [9]:
dataset_df.fillna(0, inplace = True)

In [10]:
num_missing_vals_series = (dataset_df.isnull().sum(axis = 0)/dataset_df.shape[0]) * 100
num_missing_vals_series

stock_id                   0.0
date_id                    0.0
seconds_in_bucket          0.0
imbalance_size             0.0
imbalance_buy_sell_flag    0.0
reference_price            0.0
matched_size               0.0
far_price                  0.0
near_price                 0.0
bid_price                  0.0
bid_size                   0.0
ask_price                  0.0
ask_size                   0.0
wap                        0.0
target                     0.0
time_id                    0.0
row_id                     0.0
dtype: float64

# 3) Reshaping and preparing data

In [11]:
dataset_reshaped_df = dataset_df
dataset_reshaped_df.sort_values(by = ['stock_id', 'date_id', 'seconds_in_bucket'], inplace = True)

In [12]:
def compute_features(df):
    
    df['imbalance_size'] = df['imbalance_size'] * df['imbalance_buy_sell_flag']

def compute_rollover_features(df):
    
    df['prev_target'] = df.groupby('stock_id')['target'].shift(1)
    df.fillna(0, inplace = True)
    
    return df

def standardize_features(df, mu_dict=None, sigma_dict=None):
    
    if (mu_dict is None) and (sigma_dict is None):
        
        mu_dict = {}
        sigma_dict = {}
        
        mu_dict['imbalance_size'] = [0]*200
        mu_dict['reference_price'] = [0]*200
        mu_dict['matched_size'] = [0]*200
        mu_dict['far_price'] = [0]*200
        mu_dict['near_price'] = [0]*200
        mu_dict['bid_price'] = [0]*200
        mu_dict['bid_size'] = [0]*200
        mu_dict['ask_price'] = [0]*200
        mu_dict['ask_size'] = [0]*200
        mu_dict['wap'] = [0]*200
        mu_dict['prev_target'] = [0]*200
        
        sigma_dict['imbalance_size'] = [0]*200
        sigma_dict['reference_price'] = [0]*200
        sigma_dict['matched_size'] = [0]*200
        sigma_dict['far_price'] = [0]*200
        sigma_dict['near_price'] = [0]*200
        sigma_dict['bid_price'] = [0]*200
        sigma_dict['bid_size'] = [0]*200
        sigma_dict['ask_price'] = [0]*200
        sigma_dict['ask_size'] = [0]*200
        sigma_dict['wap'] = [0]*200
        sigma_dict['prev_target'] = [0]*200
        
        for stock_id in range(df['stock_id'].max()+1):
            
            mu_dict['imbalance_size'][stock_id] = df[df['stock_id'] == stock_id]['imbalance_size'].mean()
            mu_dict['reference_price'][stock_id] = df[df['stock_id'] == stock_id]['reference_price'].mean()
            mu_dict['matched_size'][stock_id] = df[df['stock_id'] == stock_id]['matched_size'].mean()
            mu_dict['far_price'][stock_id] = df[df['stock_id'] == stock_id]['far_price'].mean()
            mu_dict['near_price'][stock_id] = df[df['stock_id'] == stock_id]['near_price'].mean()
            mu_dict['bid_price'][stock_id] = df[df['stock_id'] == stock_id]['bid_price'].mean()
            mu_dict['bid_size'][stock_id] = df[df['stock_id'] == stock_id]['bid_size'].mean()
            mu_dict['ask_price'][stock_id] = df[df['stock_id'] == stock_id]['ask_price'].mean()
            mu_dict['ask_size'][stock_id] = df[df['stock_id'] == stock_id]['ask_size'].mean()
            mu_dict['wap'][stock_id] = df[df['stock_id'] == stock_id]['wap'].mean()
            mu_dict['prev_target'][stock_id] = df[df['stock_id'] == stock_id]['prev_target'].mean()
            
            sigma_dict['imbalance_size'][stock_id] = df[df['stock_id'] == stock_id]['imbalance_size'].std()
            sigma_dict['reference_price'][stock_id] = df[df['stock_id'] == stock_id]['reference_price'].std()
            sigma_dict['matched_size'][stock_id] = df[df['stock_id'] == stock_id]['matched_size'].std()
            sigma_dict['far_price'][stock_id] = df[df['stock_id'] == stock_id]['far_price'].std()
            sigma_dict['near_price'][stock_id] = df[df['stock_id'] == stock_id]['near_price'].std()
            sigma_dict['bid_price'][stock_id] = df[df['stock_id'] == stock_id]['bid_price'].std()
            sigma_dict['bid_size'][stock_id] = df[df['stock_id'] == stock_id]['bid_size'].std()
            sigma_dict['ask_price'][stock_id] = df[df['stock_id'] == stock_id]['ask_price'].std()
            sigma_dict['ask_size'][stock_id] = df[df['stock_id'] == stock_id]['ask_size'].std()
            sigma_dict['wap'][stock_id] = df[df['stock_id'] == stock_id]['wap'].std()
            sigma_dict['prev_target'][stock_id] = df[df['stock_id'] == stock_id]['prev_target'].std()
    
    df['imbalance_size'] = (df['imbalance_size'] - df['stock_id'].map(lambda x: mu_dict['imbalance_size'][x]))/df['stock_id'].map(lambda x: sigma_dict['imbalance_size'][x])
    df['reference_price'] = (df['reference_price'] - df['stock_id'].map(lambda x: mu_dict['reference_price'][x]))/df['stock_id'].map(lambda x: sigma_dict['reference_price'][x])
    df['matched_size'] = (df['matched_size'] - df['stock_id'].map(lambda x: mu_dict['matched_size'][x]))/df['stock_id'].map(lambda x: sigma_dict['matched_size'][x])
    df['far_price'] = (df['far_price'] - df['stock_id'].map(lambda x: mu_dict['far_price'][x]))/df['stock_id'].map(lambda x: sigma_dict['far_price'][x])
    df['near_price'] = (df['near_price'] - df['stock_id'].map(lambda x: mu_dict['near_price'][x]))/df['stock_id'].map(lambda x: sigma_dict['near_price'][x])
    df['bid_price'] = (df['bid_price'] - df['stock_id'].map(lambda x: mu_dict['bid_price'][x]))/df['stock_id'].map(lambda x: sigma_dict['bid_price'][x])
    df['bid_size'] = (df['bid_size'] - df['stock_id'].map(lambda x: mu_dict['bid_size'][x]))/df['stock_id'].map(lambda x: sigma_dict['bid_size'][x])
    df['ask_price'] = (df['ask_price'] - df['stock_id'].map(lambda x: mu_dict['ask_price'][x]))/df['stock_id'].map(lambda x: sigma_dict['ask_price'][x])
    df['ask_size'] = (df['ask_size'] - df['stock_id'].map(lambda x: mu_dict['ask_size'][x]))/df['stock_id'].map(lambda x: sigma_dict['ask_size'][x])    
    df['wap'] = (df['wap'] - df['stock_id'].map(lambda x: mu_dict['wap'][x]))/df['stock_id'].map(lambda x: sigma_dict['wap'][x])        
        
    df['prev_target'] = (df['prev_target'] - df['stock_id'].map(lambda x: mu_dict['prev_target'][x]))/df['stock_id'].map(lambda x: sigma_dict['prev_target'][x])
    
    return (mu_dict, sigma_dict)
   
def drop_features(df):
    
    #Drop the following features.
    df.drop(['row_id', 'imbalance_buy_sell_flag'], axis = 1, inplace = True)
    
    if 'currently_scored' in df.columns:
        df.drop(['currently_scored'], axis = 1, inplace = True)

In [13]:
#Recompute some features.
compute_features(dataset_reshaped_df)

#Compute roll-over features.
dataset_reshaped_df = compute_rollover_features(dataset_reshaped_df)

#Standardize features.
mu_dict, sigma_dict = standardize_features(dataset_reshaped_df)

#Drop features from the dataset.
drop_features(dataset_reshaped_df)

In [14]:
dataset_reshaped_df[dataset_reshaped_df['stock_id'] == 0].head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,prev_target
0,0,0,0,0.420313,-0.013518,-0.587909,-0.906572,-0.912704,0.045702,0.326687,0.042624,-0.517844,0.094030,-3.029704,0,0.043116
191,0,0,10,0.218021,0.113973,-0.498553,-0.906572,-0.912704,0.045702,-0.292750,0.042624,-0.257284,0.029652,0.389814,1,-0.457311
382,0,0,20,0.218021,0.050228,-0.498553,-0.906572,-0.912704,0.045702,-0.416637,-0.021117,-0.454750,-0.000153,4.220009,2,0.107503
573,0,0,30,0.218021,0.177718,-0.498553,-0.906572,-0.912704,0.173495,0.264903,0.106365,0.136070,0.144697,5.450249,3,0.740151
764,0,0,40,0.209248,0.369550,-0.494678,-0.906572,-0.912704,0.301885,-0.283994,0.298183,-0.203679,0.282991,3.169775,4,0.943354


In [15]:
train_df = dataset_reshaped_df[dataset_reshaped_df['date_id'] < 478]
val_df = dataset_reshaped_df[dataset_reshaped_df['date_id'] >= 478]

In [16]:
def get_training_data(stock_id, test_val_data_dates, threshold = 0.9):
    
    #Get all training data associated with the stock_id.
    stock_train_df = train_df[train_df['stock_id'] == stock_id].copy()
    
    #Drop columns not relevant for the cosine similarity and convert the result to arrays.
    train_curr_date_array = stock_train_df.drop(['stock_id', 'target', 'date_id', 'time_id', 'seconds_in_bucket'], axis = 1).values
    train_target_date_array = (stock_train_df['target'].values).reshape(-1, 1)
    
    indices_of_closest_arrays = None
    
    for test_val_data_date in test_val_data_dates:
        
        #print(train_curr_date_array.shape, test_val_data_date.shape)
    
        #Compute the cosine similarity between the single row of test data and the all the rows of training data.
        cos_sim = cosine_similarity(train_curr_date_array, test_val_data_date.reshape(1, -1))
        
        if indices_of_closest_arrays is None:
            #Determine the indices of training data with higher cosine similarity than the threshold.
            indices_of_closest_arrays = np.where(cos_sim > threshold)[0]
        
        else:
            indices_of_closest_arrays = np.concatenate((indices_of_closest_arrays, np.where(cos_sim > threshold)[0]))
    
    indices_of_closest_arrays = np.unique(indices_of_closest_arrays)
    indices_of_closest_arrays = np.sort(indices_of_closest_arrays)
    
    #Get training rows corressponding to the indices determined earlier.
    ret_train_arr = train_curr_date_array[indices_of_closest_arrays]
    ret_train_arr = ret_train_arr.reshape(1, ret_train_arr.shape[0], ret_train_arr.shape[1])
    
    #Get target values for the training rows determined earlier.
    ret_train_tar_arr = train_target_date_array[indices_of_closest_arrays]
    ret_train_tar_arr = ret_train_tar_arr.reshape(1, ret_train_tar_arr.shape[0], ret_train_tar_arr.shape[1])
    
    return ret_train_arr, ret_train_tar_arr

#& (val_df['seconds_in_bucket'] == 0)
tr_array = get_training_data(2, val_df[(val_df['stock_id'] == 2) & (val_df['date_id'] == 478)].drop(['stock_id', 'target', 'date_id', 'time_id', 'seconds_in_bucket', 'target'], axis = 1).values, threshold = 0.9)
print(tr_array[0].shape, tr_array[1].shape)

(1, 7746, 11) (1, 7746, 1)


In [17]:
def get_valid_test_data(stock_id, seq_len = 100):
    
    test_val_df = val_df[val_df['stock_id'] == stock_id].copy()
    test_val_df.drop(['stock_id', 'date_id', 'time_id', 'seconds_in_bucket'], inplace = True, axis = 1)
    
    val_target_array = None
    
    if 'target' in test_val_df.columns:
        val_target_array = (test_val_df['target'].values).reshape(-1, 1)
        test_val_df.drop(['target'], inplace = True, axis = 1)
    
    val_array = test_val_df.values
    val_target_array = val_target_array
    
    return val_array, val_target_array

tr_array = get_valid_test_data(0, seq_len = 2000)
print(tr_array[0].shape, tr_array[0].shape)

(165, 11) (165, 11)


# 4) Creating models specific to each stock ID

In [18]:
#Function declarations
def step_decay(epoch, learning_rate):
    # initialize the base initial learning rate, drop factor, and epochs to drop every
    init_lr = 1
    factor = 0.9
    drop_every = 6
    # compute learning rate for the current epoch
    learning_rate = init_lr*(factor ** (np.floor(epoch) / drop_every))
    return learning_rate

In [19]:
#Input dimension - 1 has been added because 'PREV_TARGET field is introduced much later'
INPUT_SIZE = 11
print(f'Input size = {INPUT_SIZE}')

#Declaring a dictionary of models - a model for each stock ID.
models_dict = {}

Input size = 11


In [20]:
def train_models(threshold):
    
    early_checkpoint = EarlyStopping(patience=2, monitor='mae', mode='min')

    seed_value = 42
    tf.random.set_seed(seed_value)

    #Construct train/validation sets separately for all stock IDs.
    for curr_stock_id in dataset_reshaped_df['stock_id'].unique():
        
        print(f'Training model for stock ID {curr_stock_id}')
        
        curr_train_mae = 0
        curr_val_mae = 0
        
        #Get validation data (numpy array) for the current stock_id.
        curr_stock_val_data = get_valid_test_data(curr_stock_id, INPUT_SIZE)
        
        #Construct a GRU model for the current stock ID.
        model = tf.keras.models.Sequential([
            tf.keras.layers.GRU(8, return_sequences=True, input_shape=(None, INPUT_SIZE)),
            tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation='linear'))])
        
        #Compile the model.
        lr_scheduler = keras.callbacks.LearningRateScheduler(step_decay)
        opt = tf.keras.optimizers.Adam(learning_rate=0.01)
        model.compile(loss="mean_absolute_error", optimizer=opt, metrics=["mae"])
        
        #Get training data (numpy array) corressponding to the current test row.
        curr_test_row = val_df[val_df['stock_id'] == curr_stock_id].drop(['stock_id', 'target', 'date_id', 'time_id', 'seconds_in_bucket', 'target'], axis = 1).values
        curr_stock_train_data = get_training_data(curr_stock_id, curr_test_row, threshold = threshold)    
        
        if curr_stock_train_data[0].shape[1] == 0:
            curr_stock_train_data = get_training_data(curr_stock_id, curr_test_row, threshold = 0.8)
        elif curr_stock_train_data[0].shape[1] == 0:
            curr_stock_train_data = get_training_data(curr_stock_id, curr_test_row, threshold = 0.7)
        else:
            curr_stock_train_data = get_training_data(curr_stock_id, curr_test_row, threshold = 0.6)
        
        #Create training dataset.
        train_dataset = tf.data.Dataset.from_tensor_slices((curr_stock_train_data[0], curr_stock_train_data[1]))
        train_dataset = train_dataset.batch(500)
        
        #Create validation dataset.
        val_dataset = tf.data.Dataset.from_tensor_slices((curr_stock_val_data[0].reshape(1, 165, 11), curr_stock_val_data[1].reshape(1, 165, 1)))
        val_dataset = val_dataset.batch(500)
        
        #Train the model.
        history = model.fit(train_dataset, validation_data = val_dataset, epochs=20, callbacks=[early_checkpoint, lr_scheduler], verbose = 1)
            
        #Determine the cumulative training and validation MAEs across the different rows of the validation
        #data.
        curr_train_mae = history.history['mae'][-1]
        curr_val_mae = history.history['val_mae'][-1]
        
        print(f'Stock ID = {curr_stock_id}, Threshold = {threshold}, Training MAE = {curr_train_mae}, Validation MAE = {curr_val_mae}')
        
        if curr_stock_id not in models_dict.keys():
            models_dict[curr_stock_id] = [model, threshold, curr_train_mae, curr_val_mae]
        
        else:
            if models_dict[curr_stock_id][3] > curr_val_mae:
                models_dict[curr_stock_id][0] = model
                models_dict[curr_stock_id][1] = threshold   
                models_dict[curr_stock_id][2] = curr_train_mae
                models_dict[curr_stock_id][3] = curr_val_mae   

In [ ]:
%%time

#threshold_list = [0.95, 0.9, 0.8, 0.7] 
threshold_list = [0.90]

for threshold in threshold_list:

    print('Cosine similarity threshold = ', threshold)
    train_models(threshold)

Cosine similarity threshold =  0.9
Training model for stock ID 0
Epoch 1/20
1/1 [==============================] - 13s 13s/step - loss: 4.3666 - mae: 4.3666 - val_loss: 4.9454 - val_mae: 4.9454 - lr: 1.0000
Epoch 2/20
1/1 [==============================] - 11s 11s/step - loss: 5.7356 - mae: 5.7356 - val_loss: 4.7486 - val_mae: 4.7486 - lr: 0.9826
Epoch 3/20
1/1 [==============================] - 10s 10s/step - loss: 5.3403 - mae: 5.3403 - val_loss: 4.4338 - val_mae: 4.4338 - lr: 0.9655
Stock ID = 0, Threshold = 0.9, Training MAE = 5.340296745300293, Validation MAE = 4.433807849884033
Training model for stock ID 1
Epoch 1/20
1/1 [==============================] - 9s 9s/step - loss: 8.7560 - mae: 8.7560 - val_loss: 6.1578 - val_mae: 6.1578 - lr: 1.0000
Epoch 2/20
1/1 [==============================] - 8s 8s/step - loss: 9.1103 - mae: 9.1103 - val_loss: 5.7446 - val_mae: 5.7446 - lr: 0.9826
Epoch 3/20
1/1 [==============================] - 8s 8s/step - loss: 8.5080 - mae: 8.5080 - val_los

1/1 [==============================] - 9s 9s/step - loss: 6.3113 - mae: 6.3113 - val_loss: 7.6085 - val_mae: 7.6085 - lr: 0.9826
Epoch 3/20
1/1 [==============================] - 7s 7s/step - loss: 7.0647 - mae: 7.0647 - val_loss: 5.8355 - val_mae: 5.8355 - lr: 0.9655
Epoch 4/20
1/1 [==============================] - 5s 5s/step - loss: 6.0696 - mae: 6.0696 - val_loss: 5.5252 - val_mae: 5.5252 - lr: 0.9487
Epoch 5/20
1/1 [==============================] - 6s 6s/step - loss: 5.7828 - mae: 5.7828 - val_loss: 5.1764 - val_mae: 5.1764 - lr: 0.9322
Epoch 6/20
1/1 [==============================] - 5s 5s/step - loss: 6.1290 - mae: 6.1290 - val_loss: 5.3018 - val_mae: 5.3018 - lr: 0.9159
Epoch 7/20
1/1 [==============================] - 5s 5s/step - loss: 6.1165 - mae: 6.1165 - val_loss: 5.4701 - val_mae: 5.4701 - lr: 0.9000
Stock ID = 6, Threshold = 0.9, Training MAE = 6.116473197937012, Validation MAE = 5.470073699951172
Training model for stock ID 7
Epoch 1/20
1/1 [=========================

Epoch 3/20
1/1 [==============================] - 6s 6s/step - loss: 5.3124 - mae: 5.3124 - val_loss: 3.8500 - val_mae: 3.8500 - lr: 0.9655
Stock ID = 14, Threshold = 0.9, Training MAE = 5.312409400939941, Validation MAE = 3.8499858379364014
Training model for stock ID 15
Epoch 1/20
1/1 [==============================] - 12s 12s/step - loss: 5.8475 - mae: 5.8475 - val_loss: 5.8241 - val_mae: 5.8241 - lr: 1.0000
Epoch 2/20
1/1 [==============================] - 10s 10s/step - loss: 6.6811 - mae: 6.6811 - val_loss: 5.3181 - val_mae: 5.3181 - lr: 0.9826
Epoch 3/20
1/1 [==============================] - 9s 9s/step - loss: 5.8823 - mae: 5.8823 - val_loss: 5.1739 - val_mae: 5.1739 - lr: 0.9655
Stock ID = 15, Threshold = 0.9, Training MAE = 5.882294654846191, Validation MAE = 5.173861980438232
Training model for stock ID 16
Epoch 1/20
1/1 [==============================] - 13s 13s/step - loss: 6.3138 - mae: 6.3138 - val_loss: 5.8827 - val_mae: 5.8827 - lr: 1.0000
Epoch 2/20
1/1 [=============

Epoch 9/20
1/1 [==============================] - 7s 7s/step - loss: 6.6536 - mae: 6.6536 - val_loss: 5.4295 - val_mae: 5.4295 - lr: 0.8689
Epoch 10/20
1/1 [==============================] - 6s 6s/step - loss: 6.6296 - mae: 6.6296 - val_loss: 5.4401 - val_mae: 5.4401 - lr: 0.8538
Epoch 11/20
1/1 [==============================] - 6s 6s/step - loss: 6.6969 - mae: 6.6969 - val_loss: 5.5811 - val_mae: 5.5811 - lr: 0.8390
Epoch 12/20
1/1 [==============================] - 6s 6s/step - loss: 6.6972 - mae: 6.6972 - val_loss: 5.9227 - val_mae: 5.9227 - lr: 0.8243
Stock ID = 19, Threshold = 0.9, Training MAE = 6.6972479820251465, Validation MAE = 5.922667026519775
Training model for stock ID 20
Epoch 1/20
1/1 [==============================] - 7s 7s/step - loss: 5.5065 - mae: 5.5065 - val_loss: 5.3651 - val_mae: 5.3651 - lr: 1.0000
Epoch 2/20
1/1 [==============================] - 5s 5s/step - loss: 5.9440 - mae: 5.9440 - val_loss: 6.0492 - val_mae: 6.0492 - lr: 0.9826
Epoch 3/20
1/1 [========

Epoch 6/20
1/1 [==============================] - 6s 6s/step - loss: 6.0141 - mae: 6.0141 - val_loss: 4.6802 - val_mae: 4.6802 - lr: 0.9159
Epoch 7/20
1/1 [==============================] - 12s 12s/step - loss: 5.6237 - mae: 5.6237 - val_loss: 3.9288 - val_mae: 3.9288 - lr: 0.9000
Epoch 8/20
1/1 [==============================] - 6s 6s/step - loss: 5.3000 - mae: 5.3000 - val_loss: 3.3883 - val_mae: 3.3883 - lr: 0.8843
Epoch 9/20
1/1 [==============================] - 6s 6s/step - loss: 5.0451 - mae: 5.0451 - val_loss: 3.4029 - val_mae: 3.4029 - lr: 0.8689
Epoch 10/20
1/1 [==============================] - 6s 6s/step - loss: 4.9252 - mae: 4.9252 - val_loss: 3.3818 - val_mae: 3.3818 - lr: 0.8538
Epoch 11/20
1/1 [==============================] - 9s 9s/step - loss: 4.8657 - mae: 4.8657 - val_loss: 3.3623 - val_mae: 3.3623 - lr: 0.8390
Epoch 12/20
1/1 [==============================] - 12s 12s/step - loss: 4.8397 - mae: 4.8397 - val_loss: 3.4158 - val_mae: 3.4158 - lr: 0.8243
Epoch 13/20
1

1/1 [==============================] - 10s 10s/step - loss: 3.8777 - mae: 3.8777 - val_loss: 2.9414 - val_mae: 2.9414 - lr: 0.9322
Epoch 6/20
1/1 [==============================] - 10s 10s/step - loss: 3.5787 - mae: 3.5787 - val_loss: 3.1058 - val_mae: 3.1058 - lr: 0.9159
Epoch 7/20
1/1 [==============================] - 8s 8s/step - loss: 3.5506 - mae: 3.5506 - val_loss: 3.2291 - val_mae: 3.2291 - lr: 0.9000
Epoch 8/20
1/1 [==============================] - 8s 8s/step - loss: 3.6939 - mae: 3.6939 - val_loss: 3.1782 - val_mae: 3.1782 - lr: 0.8843
Epoch 9/20
1/1 [==============================] - 10s 10s/step - loss: 3.6635 - mae: 3.6635 - val_loss: 3.2350 - val_mae: 3.2350 - lr: 0.8689
Stock ID = 35, Threshold = 0.9, Training MAE = 3.663515567779541, Validation MAE = 3.235016107559204
Training model for stock ID 36
Epoch 1/20
1/1 [==============================] - 8s 8s/step - loss: 5.6226 - mae: 5.6226 - val_loss: 5.1356 - val_mae: 5.1356 - lr: 1.0000
Epoch 2/20
1/1 [=================

In [ ]:
print('The MAEs of the models are: -')
count1 = 0
count2 = 0
for curr_stock_id in models_dict.keys():
    print(f'Stock ID = {curr_stock_id}, Cosine similarity threshold = {models_dict[curr_stock_id][1]}, Training MAE = {models_dict[curr_stock_id][2]}, Validation MAE = {models_dict[curr_stock_id][3]}')

    #Check the number of stocks predictions for which have an MAE of less than 5.
    if models_dict[curr_stock_id][3] < 4:
        count1 += 1

    if models_dict[curr_stock_id][3] < 5:
        count2 += 1

In [ ]:
print('Number of stocks predictions for which have an MAE of less than 4 = ', count1)
print('Number of stocks predictions for which have an MAE of less than 5 = ', count2)